In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import voeventcache
import sqlalchemy

In [ ]:
import logging
logging.basicConfig()

In [ ]:
from voeventcache.tests.resources import swift_bat_grb_pos_v2_etree
from voeventcache.database.models import Voevent, Base
row = Voevent.from_etree(swift_bat_grb_pos_v2_etree)

In [ ]:
from voeventcache.database import db_utils
from voeventcache.tests.config import testdb_scratch_url, admin_db_url
if not db_utils.check_database_exists(testdb_scratch_url):
    db_utils.create_database(admin_db_url, testdb_scratch_url.database)
engine = sqlalchemy.engine.create_engine(testdb_scratch_url)
Base.metadata.create_all(engine)
engine.dispose()

In [ ]:
import sqlalchemy
import sqlalchemy.orm as orm
engine = sqlalchemy.engine.create_engine(testdb_scratch_url)
Session = orm.sessionmaker(bind=engine)
s = Session()
s.query(Voevent).first()

In [ ]:
import voeventcache.tests.fixtures.fake as fake
from datetime import datetime, timedelta
start = datetime(2015, 1, 1)
interval = timedelta(minutes=15)
n_interval = 4*6
packets = fake.heartbeat_packets(start,
                       start+n_interval*interval,
                       interval)
s.add_all((Voevent.from_etree(p) for p in packets))
s.commit()
s.query(Voevent).count()